In [1]:
# from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2 ##STrAdaBoost.R2
# from TwoStageTrAdaBoostR2 import TwoStageTrAdaBoostR2 ##two-stage TrAdaBoost.R2

import pandas as pd
import sys
import numpy as np
from pandas import DataFrame
from sklearn.model_selection import train_test_split, KFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.cluster import KMeans

import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import seaborn as sns

from scipy.stats.stats import pearsonr
from math import sqrt
import statistics 
from scipy.stats import *
from scipy.spatial import distance

######### Instance Transfer repositories ####################
from adapt.instance_based import TrAdaBoost, TrAdaBoostR2, TwoStageTrAdaBoostR2
from sklearn.model_selection import GridSearchCV
from adapt.instance_based import KMM

print("Repositories uploaded!!")

Repositories uploaded!!


In [2]:
################################### Concrete ###########################################################################################################
ConcreteData_df = pd.read_excel('UCI_regression/Concrete/Concrete_Data.xls') ## 'Cement' found to be correlated at 0.4 :: 100
print("Concrete Data")
print("-------------------------------------------")
print(ConcreteData_df.shape)


##############################################################################################
# concrete_cols = ConcreteData_df.columns
# ss = StandardScaler()
# ConcreteData_df[concrete_cols] = ss.fit_transform(ConcreteData_df[concrete_cols])
# print(ConcreteData_df)


drop_col_concrete = ['Cement']

concrete_tgt_df = ConcreteData_df.loc[(ConcreteData_df['Cement'] <= 225)]
concrete_tgt_df = concrete_tgt_df.drop(drop_col_concrete, axis = 1)
concrete_tgt_df = concrete_tgt_df.reset_index(drop=True)
print("Target : ",concrete_tgt_df.shape)

concrete_source1_df = ConcreteData_df.loc[(ConcreteData_df['Cement'] > 225) & (ConcreteData_df['Cement'] <= 350)]
concrete_source1_df = concrete_source1_df.drop(drop_col_concrete, axis = 1)
concrete_source1_df = concrete_source1_df.reset_index(drop=True)
print("Source 1: ",concrete_source1_df.shape)

concrete_source2_df = ConcreteData_df.loc[(ConcreteData_df['Cement'] > 350)]
concrete_source2_df = concrete_source2_df.drop(drop_col_concrete, axis = 1)
concrete_source2_df = concrete_source2_df.reset_index(drop=True)
print("Source 2: ",concrete_source2_df.shape)

################################# Standardization ############################
from sklearn.preprocessing import StandardScaler
concrete_cols = concrete_tgt_df.columns.difference(['ConcreteCompressiveStrength'])

ss = StandardScaler()
concrete_tgt_df[concrete_cols] = ss.fit_transform(concrete_tgt_df[concrete_cols])
concrete_source1_df[concrete_cols] = ss.fit_transform(concrete_source1_df[concrete_cols])
concrete_source2_df[concrete_cols] = ss.fit_transform(concrete_source2_df[concrete_cols])


############################ Concatenating the source datasets ############################
concrete_source_df = pd.concat([concrete_source1_df, concrete_source2_df], ignore_index = True)
concrete_source_df = concrete_source_df.reset_index(drop = True)


#################### Splitting into features and target ####################
target_concrete = ['ConcreteCompressiveStrength']

concrete_source_df_y = concrete_source_df[target_concrete]
concrete_source_df_X = concrete_source_df.drop(target_concrete, axis = 1)

features_concrete = concrete_source_df_X.columns
################################################################################################################################################################

Concrete Data
-------------------------------------------
(1030, 9)
Target :  (368, 8)
Source 1:  (406, 8)
Source 2:  (256, 8)


In [50]:
########################### TwoStagetrAdaBoostR2 Concrete #######################################

from adapt.instance_based import TrAdaBoost, TrAdaBoostR2, TwoStageTrAdaBoostR2

def get_estimator(**kwargs):
    return DecisionTreeRegressor(max_depth = 6)

kwargs_TwoTrAda = {'steps': 30,
                    'fold': 10,
                  'learning_rate': 0.1}

print("Specification for TrAdaboost.R2 requirement complete!")

##################################################################################################

print("Two-Stage TrAdaboost.R2")
print("-------------------------------------------")

r2scorelist_TwoTrAda_concrete = []
rmselist_TwoTrAda_concrete = []

kf = KFold(n_splits = 50)

for train_idx, test_idx in kf.split(concrete_tgt_df):
        
    concrete_train_df_X = concrete_tgt_df.iloc[train_idx].loc[:, features_concrete]
    concrete_test_df_X = concrete_tgt_df.iloc[test_idx][features_concrete]
    concrete_train_df_y = concrete_tgt_df.iloc[train_idx].loc[:,target_concrete]
    concrete_test_df_y = concrete_tgt_df.loc[test_idx][target_concrete]
        
    concrete_X_df = pd.concat([concrete_source_df_X, concrete_train_df_X], ignore_index=True)
    concrete_y_df = pd.concat([concrete_source_df_y, concrete_train_df_y], ignore_index=True)

    concrete_np_train_X = concrete_X_df.to_numpy()
    concrete_np_train_y = concrete_y_df.to_numpy()

    concrete_np_test_X = concrete_test_df_X.to_numpy()
    concrete_np_test_y = concrete_test_df_y.to_numpy()

    concrete_np_train_y_list = concrete_np_train_y.ravel()
    concrete_np_test_y_list = concrete_np_test_y.ravel()

    src_size_concrete = len(concrete_source_df_y)
    tgt_size_concrete = len(concrete_train_df_y)
    
    src_idx = np.arange(start = 0, stop = (src_size_concrete - 1), step=1)
    tgt_idx = np.arange(start = src_size_concrete, stop = ((src_size_concrete + tgt_size_concrete)-1), step=1)


    model_TwoTrAda_concrete = TwoStageTrAdaBoostR2(get_estimator = get_estimator, n_estimators = 100, cv = 10) 
    model_TwoTrAda_concrete.fit(concrete_np_train_X, concrete_np_train_y_list, src_idx, tgt_idx)

    y_pred_TwoTrAda_concrete = model_TwoTrAda_concrete.predict(concrete_np_test_X)
    
    mse_TwoTrAda_concrete = sqrt(mean_squared_error(concrete_np_test_y, y_pred_TwoTrAda_concrete))
    rmselist_TwoTrAda_concrete.append(mse_TwoTrAda_concrete)
        
    r2_score_TwoTrAda_concrete = pearsonr(concrete_np_test_y_list, y_pred_TwoTrAda_concrete)
    r2_score_TwoTrAda_concrete = (r2_score_TwoTrAda_concrete[0])**2
    r2scorelist_TwoTrAda_concrete.append(r2_score_TwoTrAda_concrete)


print("RMSE List of Two-Stage TrAdaboost.R2:", rmselist_TwoTrAda_concrete)
print("R^2 List of TrAdaboostR2:", r2scorelist_TwoTrAda_concrete)

print("\n")

print("Mean, STDev of RMSE:", statistics.mean(rmselist_TwoTrAda_concrete), statistics.stdev(rmselist_TwoTrAda_concrete))
print("Mean, STDev of R^2:", statistics.mean(r2scorelist_TwoTrAda_concrete), statistics.stdev(r2scorelist_TwoTrAda_concrete))


print("-------------------------------------------")

Specification for TrAdaboost.R2 requirement complete!
Two-Stage TrAdaboost.R2
-------------------------------------------
Binary search has not converged. Set value to the current best.
RMSE List of Two-Stage TrAdaboost.R2: [5.809211408696263, 7.789066720392749, 8.957489047138598, 8.263744616657963, 8.760590871962252, 6.889771112642312, 6.085981882299316, 5.799087003967696, 5.602282791650463, 5.631358842544836]
R^2 List of TrAdaboostR2: [0.8120508206082601, 0.6500653970810222, 0.7118824741791502, 0.6601635119289769, 0.7906092117479931, 0.6802344210649145, 0.797685700198735, 0.7235199217672004, 0.5507047981805997, 0.6609043092340013]
Mean, STDev of RMSE: 6.958858429795245 1.360912074806487
Mean, STDev of R^2: 0.7037820565990853 0.0810677842927172
-------------------------------------------


In [3]:
#################################### STrAdaBoost.R2 Active Sampling Concrete ################################################################
from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2

n_estimators = 500
steps = 30
fold = 20

r2scorelist_stradaboost_concrete = []
rmselist_stradaboost_concrete = []

print("STrAdaboost.R2")
print("-------------------------------------------")


kf = KFold(n_splits = 10)

for train_idx, test_idx in kf.split(concrete_tgt_df):
        
    concrete_train_df_X = concrete_tgt_df.iloc[train_idx].loc[:, features_concrete]
    concrete_test_df_X = concrete_tgt_df.iloc[test_idx][features_concrete]
    concrete_train_df_y = concrete_tgt_df.iloc[train_idx].loc[:,target_concrete]
    concrete_test_df_y = concrete_tgt_df.loc[test_idx][target_concrete]
        
    concrete_X_df = pd.concat([concrete_source_df_X, concrete_train_df_X], ignore_index=True)
    concrete_y_df = pd.concat([concrete_source_df_y, concrete_train_df_y], ignore_index=True)

    concrete_np_train_X = concrete_X_df.to_numpy()
    concrete_np_train_y = concrete_y_df.to_numpy()

    concrete_np_test_X = concrete_test_df_X.to_numpy()
    concrete_np_test_y = concrete_test_df_y.to_numpy()

    concrete_np_train_y_list = concrete_np_train_y.ravel()
    concrete_np_test_y_list = concrete_np_test_y.ravel()
    
    sample_size = [len(concrete_source_df_X), len(concrete_train_df_X)]


    model_stradaboost_concrete = TwoStageTrAdaBoostR2(DecisionTreeRegressor(max_depth = 2),
                        n_estimators = n_estimators, sample_size = sample_size,
                        steps = steps, fold = fold)


    model_stradaboost_concrete.fit(concrete_np_train_X, concrete_np_train_y_list)
    y_pred_stradaboost_concrete = model_stradaboost_concrete.predict(concrete_np_test_X)


    mse_stradaboost_concrete = sqrt(mean_squared_error(concrete_np_test_y, y_pred_stradaboost_concrete))
    rmselist_stradaboost_concrete.append(mse_stradaboost_concrete)
        
    r2_score_stradaboost_concrete = pearsonr(concrete_np_test_y_list, y_pred_stradaboost_concrete)
    r2_score_stradaboost_concrete = (r2_score_stradaboost_concrete[0])**2
    r2scorelist_stradaboost_concrete.append(r2_score_stradaboost_concrete)


print("RMSE List of STrAdaboost.R2:", rmselist_stradaboost_concrete)
print("R^2 List of STrAdaboost.R2:", r2scorelist_stradaboost_concrete)

print("\n")

print("Mean, STDev of RMSE:", statistics.mean(rmselist_stradaboost_concrete), statistics.stdev(rmselist_stradaboost_concrete))
print("Mean, STDev of R^2:", statistics.mean(r2scorelist_stradaboost_concrete), statistics.stdev(r2scorelist_stradaboost_concrete))


STrAdaboost.R2
-------------------------------------------
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
RMSE List of STrAdaboost.R2: [6.994489101753536, 9.739851688106546, 10.457991791358877, 6.7359412350337164, 7.572827288555538, 13.058069437658203, 13.920710314947936, 12.627235382992414, 13.676418934509686, 13.057758501045207]
R^2 List of STrAdaboost.R2: [0.8951031551637316, 0.5707872204610716, 0.5739619159904189, 0.778277707728226, 0.7821824942290457, 0.772774331163075, 0.7475361532645384, 0.4650169774171834, 0.4759976066566447, 0.48815597424931545]


Mean, STDev of RMSE: 10.784129367596165 2.8712996433156035
Mean, STDev of R^2: 0.6549793536323251 0.1567

In [10]:
######################### Gradient Boosting Regression Transfer Learning Concrete #######################################

from sklearn.ensemble import GradientBoostingRegressor


print("Gradient Boosting Regression Transfer Learning")
print("-------------------------------------------")

r2scorelist_GBRTL_concrete = []
rmselist_GBRTL_concrete = []

kf = KFold(n_splits = 50)

for train_idx, test_idx in kf.split(concrete_tgt_df):
        
    concrete_train_df_X = concrete_tgt_df.iloc[train_idx].loc[:, features_concrete]
    concrete_test_df_X = concrete_tgt_df.iloc[test_idx][features_concrete]
    concrete_train_df_y = concrete_tgt_df.iloc[train_idx].loc[:,target_concrete]
    concrete_test_df_y = concrete_tgt_df.loc[test_idx][target_concrete]
        
    concrete_X_df = pd.concat([concrete_source_df_X, concrete_train_df_X], ignore_index=True)
    concrete_y_df = pd.concat([concrete_source_df_y, concrete_train_df_y], ignore_index=True)

    concrete_np_train_X = concrete_X_df.to_numpy()
    concrete_np_train_y = concrete_y_df.to_numpy()

    concrete_np_test_X = concrete_test_df_X.to_numpy()
    concrete_np_test_y = concrete_test_df_y.to_numpy()

    concrete_np_train_y_list = concrete_np_train_y.ravel()
    concrete_np_test_y_list = concrete_np_test_y.ravel()


    model_GBRTL_concrete = GradientBoostingRegressor(learning_rate = 0.1, max_depth = 2, n_estimators = 500)
    model_GBRTL_concrete.fit(concrete_np_train_X, concrete_np_train_y_list)

    y_pred_GBRTL_concrete = model_GBRTL_concrete.predict(concrete_test_df_X) ##Using dataframe instead of the numpy matrix
    
    mse_GBRTL_concrete = sqrt(mean_squared_error(concrete_np_test_y, y_pred_GBRTL_concrete))
    rmselist_GBRTL_concrete.append(mse_GBRTL_concrete)
        
    r2_score_GBRTL_concrete = pearsonr(concrete_np_test_y_list, y_pred_GBRTL_concrete)
    r2_score_GBRTL_concrete = (r2_score_GBRTL_concrete[0])**2
    r2scorelist_GBRTL_concrete.append(r2_score_GBRTL_concrete)


print("RMSE List of GBRTL:", rmselist_GBRTL_concrete)
print("R^2 List of GBRTL:", r2scorelist_GBRTL_concrete)

print("\n")

print("Mean, STDev of RMSE:", statistics.mean(rmselist_GBRTL_concrete), statistics.stdev(rmselist_GBRTL_concrete))
print("Mean, STDev of R^2:", statistics.mean(r2scorelist_GBRTL_concrete), statistics.stdev(r2scorelist_GBRTL_concrete))

print("-------------------------------------------")

Gradient Boosting Regression Transfer Learning
-------------------------------------------


/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(
/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(
/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(
/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(
/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(
/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/base.py:434

KeyboardInterrupt: 

In [85]:
########################### AdaBoostR2 Transfer Learning Concrete #####################################################

from sklearn.ensemble import AdaBoostRegressor


print("Adaboost.R2 Transfer Learning")
print("-------------------------------------------")

r2scorelist_AdaTL_concrete = []
rmselist_AdaTL_concrete = []

kf = KFold(n_splits = 50)

for train_idx, test_idx in kf.split(concrete_tgt_df):
        
    concrete_train_df_X = concrete_tgt_df.iloc[train_idx].loc[:, features_concrete]
    concrete_test_df_X = concrete_tgt_df.iloc[test_idx][features_concrete]
    concrete_train_df_y = concrete_tgt_df.iloc[train_idx].loc[:,target_concrete]
    concrete_test_df_y = concrete_tgt_df.loc[test_idx][target_concrete]
        
    concrete_X_df = pd.concat([concrete_source_df_X, concrete_train_df_X], ignore_index=True)
    concrete_y_df = pd.concat([concrete_source_df_y, concrete_train_df_y], ignore_index=True)

    concrete_np_train_X = concrete_X_df.to_numpy()
    concrete_np_train_y = concrete_y_df.to_numpy()

    concrete_np_test_X = concrete_test_df_X.to_numpy()
    concrete_np_test_y = concrete_test_df_y.to_numpy()

    concrete_np_train_y_list = concrete_np_train_y.ravel()
    concrete_np_test_y_list = concrete_np_test_y.ravel()


    model_AdaTL_concrete = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 2), learning_rate = 0.1, n_estimators = 500) 
    model_AdaTL_concrete.fit(concrete_np_train_X, concrete_np_train_y_list)
    
    y_pred_AdaTL_concrete = model_AdaTL_concrete.predict(concrete_np_test_X)
    
    mse_AdaTL_concrete = sqrt(mean_squared_error(concrete_np_test_y, y_pred_AdaTL_concrete))
    rmselist_AdaTL_concrete.append(mse_AdaTL_concrete)
        
    r2_score_AdaTL_concrete = pearsonr(concrete_np_test_y_list, y_pred_AdaTL_concrete)
    r2_score_AdaTL_concrete = (r2_score_AdaTL_concrete[0])**2
    r2scorelist_AdaTL_concrete.append(r2_score_AdaTL_concrete)
    

print("RMSE List of Adaboost.R2:", rmselist_AdaTL_concrete)
print("R^2 List of TrAdaboostR2:", r2scorelist_AdaTL_concrete)

print("\n")

print("Mean, STDev of RMSE:", statistics.mean(rmselist_AdaTL_concrete), statistics.stdev(rmselist_AdaTL_concrete))
print("Mean, STDev of R^2:", statistics.mean(r2scorelist_AdaTL_concrete), statistics.stdev(r2scorelist_AdaTL_concrete))


print("-------------------------------------------")

Adaboost.R2 Transfer Learning
-------------------------------------------
RMSE List of Adaboost.R2: [6.791373171158159, 5.735879589942653, 6.604514839692109, 8.559252124833652, 8.019843355779715, 9.858851603552013, 13.796013542123653, 7.861539227703065, 5.3462488854096035, 6.447350075710572, 14.29035219609247, 9.093877366294029, 8.102864183729356, 7.915139360869671, 7.922660543393983, 7.520618154620246, 9.152702540840163, 8.594937638628949, 7.127630425942871, 4.566549198087218, 5.557094722516787, 5.03665374352673, 9.234389905482134, 10.053527000685545, 11.302472253517857, 11.796576843251426, 8.750395079355743, 13.508049101580493, 8.299420347959408, 9.325731077986788, 12.187038733616513, 10.684865260740953, 12.243758715406122, 9.19033267884683, 11.43086597282712, 16.333502697811433, 9.173104088802711, 12.945723326756891, 16.862809909257386, 15.409386636873894, 14.933447806982619, 13.58094604030664, 14.161645749459796, 12.796650416594831, 18.319872811223043, 13.548391698655394, 15.832467

In [70]:
from scipy import stats

stats_StAd_rmse, pval_StAd_rmse = stats.ttest_ind(rmselist_stradaboost_concrete, rmselist_AdaTL_concrete)
print(stats_StAd_rmse, pval_StAd_rmse)

stats_StGb_rmse, pval_StGb_rmse = stats.ttest_ind(rmselist_stradaboost_concrete, rmselist_GBRTL_concrete)
print(stats_StGb_rmse, pval_StGb_rmse)

stats_StTr_rmse, pval_StTr_rmse = stats.ttest_ind(rmselist_stradaboost_concrete, rmselist_TwoTrAda_concrete)
print(stats_StTr_rmse, pval_StTr_rmse)

print("\n")

stats_StAd_r2, pval_StAd_r2 = stats.ttest_ind(r2scorelist_stradaboost_concrete, r2scorelist_AdaTL_concrete)
print(stats_StAd_r2, pval_StAd_r2)

stats_StGb_r2, pval_StGb_r2 = stats.ttest_ind(r2scorelist_stradaboost_concrete, r2scorelist_GBRTL_concrete)
print(stats_StGb_r2, pval_StGb_r2)

stats_StTr_r2, pval_StTr_r2 = stats.ttest_ind(r2scorelist_stradaboost_concrete, r2scorelist_TwoTrAda_concrete)
print(stats_StTr_r2, pval_StTr_r2)

0.06642893536867998 0.9477686137441723
1.803370278742918 0.08809539363436718
-0.6402691951701012 0.5300643497257027


0.543419163483719 0.5935081302926962
-1.064876041637561 0.30100883616380025
1.3293754446871224 0.20032802775973285


In [75]:
#### Dependent(paired) t-test
from scipy.stats import ttest_rel


stats_StAd_rmse, pval_StAd_rmse = stats.ttest_rel(rmselist_stradaboost_concrete, rmselist_AdaTL_concrete)
print(stats_StAd_rmse, pval_StAd_rmse)

stats_StGb_rmse, pval_StGb_rmse = stats.ttest_rel(rmselist_stradaboost_concrete, rmselist_GBRTL_concrete)
print(stats_StGb_rmse, pval_StGb_rmse)

stats_StTr_rmse, pval_StTr_rmse = stats.ttest_rel(rmselist_stradaboost_concrete, rmselist_TwoTrAda_concrete)
print(stats_StTr_rmse, pval_StTr_rmse)

print("\n")

stats_StAd_r2, pval_StAd_r2 = stats.ttest_rel(r2scorelist_stradaboost_concrete, r2scorelist_AdaTL_concrete)
print(stats_StAd_r2, pval_StAd_r2)

stats_StGb_r2, pval_StGb_r2 = stats.ttest_rel(r2scorelist_stradaboost_concrete, r2scorelist_GBRTL_concrete)
print(stats_StGb_r2, pval_StGb_r2)

stats_StTr_r2, pval_StTr_r2 = stats.ttest_rel(r2scorelist_stradaboost_concrete, r2scorelist_TwoTrAda_concrete)
print(stats_StTr_r2, pval_StTr_r2)

0.2664013235666798 0.7959330728063094
2.3414918477916933 0.04391284579630366
-0.745171561376906 0.47517702097403447


2.137024794237644 0.06131719383991011
-2.2358256739080944 0.052197892656036644
2.702336607860335 0.024300531692972208


In [4]:
################################### Housing ################################################################
## 'nox' found to be correlated at 0.4 :: [0.385 - 0.871] :: 50
#################################################################################################################################
HousingData_df = pd.read_csv('UCI_regression/BostonHousing/BostonHousing.csv') 
print("Housing Data")
print(HousingData_df.shape)

drop_col_housing = ['nox']
housing_tgt_df = HousingData_df.loc[(HousingData_df['nox'] <= 0.475)]
housing_tgt_df = housing_tgt_df.drop(drop_col_housing, axis = 1)
housing_tgt_df = housing_tgt_df.reset_index(drop=True)
print("Target Set: ",housing_tgt_df.shape)


housing_source1_df = HousingData_df.loc[(HousingData_df['nox'] > 0.475) & (HousingData_df['nox'] <= 0.600)]
housing_source1_df = housing_source1_df.drop(drop_col_housing, axis = 1)
housing_source1_df = housing_source1_df.reset_index(drop=True)
print("Source Set 1: ",housing_source1_df.shape)


housing_source2_df = HousingData_df.loc[(HousingData_df['nox'] > 0.600)]
housing_source2_df = housing_source2_df.drop(drop_col_housing, axis = 1)
housing_source2_df = housing_source2_df.reset_index(drop=True)
print("Source Set 2: ",housing_source2_df.shape)

################################# Standardization ############################
from sklearn.preprocessing import StandardScaler
housing_cols = housing_tgt_df.columns.difference(['medv'])

ss = StandardScaler()
housing_tgt_df[housing_cols] = ss.fit_transform(housing_tgt_df[housing_cols])
housing_source1_df[housing_cols] = ss.fit_transform(housing_source1_df[housing_cols])
housing_source2_df[housing_cols] = ss.fit_transform(housing_source2_df[housing_cols])


############################ Concatenating the source datasets ############################
housing_source_df = pd.concat([housing_source1_df, housing_source2_df], ignore_index = True)
housing_source_df = housing_source_df.reset_index(drop = True)

#################### Splitting into features and target ####################
target_housing = ['medv']

housing_source_df_y = housing_source_df[target_housing]
housing_source_df_X = housing_source_df.drop(target_housing, axis = 1)

features_housing = housing_source_df_X.columns

Housing Data
(506, 14)
Target Set:  (155, 13)
Source Set 1:  (186, 13)
Source Set 2:  (165, 13)


In [56]:
########################### TwoStagetrAdaBoostR2 Housing #######################################

from adapt.instance_based import TrAdaBoost, TrAdaBoostR2, TwoStageTrAdaBoostR2

def get_estimator(**kwargs):
    return DecisionTreeRegressor(max_depth = 6)

kwargs_TwoTrAda = {'steps': 30,
                    'fold': 10,
                  'learning_rate': 0.1}

print("Specification requirement complete!")

###########################################################################################################################

print("Two-Stage TrAdaboost.R2")
print("-------------------------------------------")

r2scorelist_TwoTrAda_housing = []
rmselist_TwoTrAda_housing = []

kf = KFold(n_splits = 10)

for train_idx, test_idx in kf.split(housing_tgt_df):
        
    housing_train_df_X = housing_tgt_df.iloc[train_idx].loc[:, features_housing]
    housing_test_df_X = housing_tgt_df.iloc[test_idx][features_housing]
    housing_train_df_y = housing_tgt_df.iloc[train_idx].loc[:,target_housing]
    housing_test_df_y = housing_tgt_df.loc[test_idx][target_housing]
    
    housing_X_df = pd.concat([housing_source_df_X, housing_train_df_X], ignore_index = True)
    housing_y_df = pd.concat([housing_source_df_y, housing_train_df_y], ignore_index = True)

    housing_np_train_X = housing_X_df.to_numpy()
    housing_np_train_y = housing_y_df.to_numpy()

    housing_np_test_X = housing_test_df_X.to_numpy()
    housing_np_test_y = housing_test_df_y.to_numpy()

    housing_np_train_y_list = housing_np_train_y.ravel()
    housing_np_test_y_list = housing_np_test_y.ravel()

    src_size_housing = len(housing_source_df_y)
    tgt_size_housing = len(housing_train_df_y)
    
    src_idx_housing = np.arange(start = 0, stop = (src_size_housing - 1), step=1)
    tgt_idx_housing = np.arange(start = src_size_housing, stop = ((src_size_housing + tgt_size_housing)-1), step = 1)


    model_TwoTrAda_housing = TwoStageTrAdaBoostR2(get_estimator = get_estimator, n_estimators = 100, cv = 10) 
    model_TwoTrAda_housing.fit(housing_np_train_X, housing_np_train_y_list, src_idx_housing, tgt_idx_housing)

    y_pred_TwoTrAda_housing = model_TwoTrAda_housing.predict(housing_np_test_X)

    mse_TwoTrAda_housing = sqrt(mean_squared_error(housing_np_test_y, y_pred_TwoTrAda_housing))
    rmselist_TwoTrAda_housing.append(mse_TwoTrAda_housing)
        
    r2_score_TwoTrAda_housing = pearsonr(housing_np_test_y_list, y_pred_TwoTrAda_housing)
    r2_score_TwoTrAda_housing = (r2_score_TwoTrAda_housing[0])**2
    r2scorelist_TwoTrAda_housing.append(r2_score_TwoTrAda_housing)


print("Mean, STDev of RMSE:", statistics.mean(rmselist_TwoTrAda_housing), statistics.stdev(rmselist_TwoTrAda_housing))
print("Mean, STDev of R^2:", statistics.mean(r2scorelist_TwoTrAda_housing), statistics.stdev(r2scorelist_TwoTrAda_housing))
print("\n")
print("RMSE List TrAdaboost.R2:", rmselist_TwoTrAda_housing)
print("R^2 List TrAdaboost.R2:", r2scorelist_TwoTrAda_housing)


print("-------------------------------------------")

Specification requirement complete!
Two-Stage TrAdaboost.R2
-------------------------------------------
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.
Binary search has not converged. Set value to the current best.


In [5]:
#################################### STrAdaBoost.R2 Active Sampling Housing ################################################################
from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2

n_estimators = 100
steps = 30
fold = 10
random_state = np.random.RandomState(1)

r2scorelist_stradaboost_housing = []
rmselist_stradaboost_housing = []

print("STrAdaboost.R2")
print("-------------------------------------------")

kf = KFold(n_splits = 10)

for train_idx, test_idx in kf.split(housing_tgt_df):
        
    housing_train_df_X = housing_tgt_df.iloc[train_idx].loc[:, features_housing]
    housing_test_df_X = housing_tgt_df.iloc[test_idx][features_housing]
    housing_train_df_y = housing_tgt_df.iloc[train_idx].loc[:,target_housing]
    housing_test_df_y = housing_tgt_df.loc[test_idx][target_housing]
    
    housing_X_df = pd.concat([housing_source_df_X, housing_train_df_X], ignore_index = True)
    housing_y_df = pd.concat([housing_source_df_y, housing_train_df_y], ignore_index = True)

    housing_np_train_X = housing_X_df.to_numpy()
    housing_np_train_y = housing_y_df.to_numpy()

    housing_np_test_X = housing_test_df_X.to_numpy()
    housing_np_test_y = housing_test_df_y.to_numpy()

    housing_np_train_y_list = housing_np_train_y.ravel()
    housing_np_test_y_list = housing_np_test_y.ravel()
    
    sample_size = [len(housing_source_df_X), len(housing_train_df_X)]


    model_stradaboost_housing = TwoStageTrAdaBoostR2(DecisionTreeRegressor(max_depth = 6),
                        n_estimators = n_estimators, sample_size = sample_size,
                        steps = steps, fold = fold, random_state = random_state)


    model_stradaboost_housing.fit(housing_np_train_X, housing_np_train_y_list)
    y_pred_stradaboost_housing = model_stradaboost_housing.predict(housing_np_test_X)


    mse_stradaboost_housing = sqrt(mean_squared_error(housing_np_test_y, y_pred_stradaboost_housing))
    rmselist_stradaboost_housing.append(mse_stradaboost_housing)
        
    r2_score_stradaboost_housing = pearsonr(housing_np_test_y_list, y_pred_stradaboost_housing)
    r2_score_stradaboost_housing = (r2_score_stradaboost_housing[0])**2
    r2scorelist_stradaboost_housing.append(r2_score_stradaboost_housing)

print("RMSE of STrAdaboost.R2:", rmselist_stradaboost_housing)
print("R^2 of STrAdaboost.R2:", r2scorelist_stradaboost_housing)

print("\n")


print("Mean, STDev of RMSE:", statistics.mean(rmselist_stradaboost_housing), statistics.stdev(rmselist_stradaboost_housing))
print("Mean, STDev of R^2:", statistics.mean(r2scorelist_stradaboost_housing), statistics.stdev(r2scorelist_stradaboost_housing))


STrAdaboost.R2
-------------------------------------------
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
RMSE of STrAdaboost.R2: [2.643518097047088, 2.0446110978325023, 1.3264814902392026, 3.1412323088120258, 6.055358754659789, 2.107677222789961, 7.369388206228197, 2.616042210901775, 3.104185474452117, 1.7473932471670033]
R^2 of STrAdaboost.R2: [0.929416103939321, 0.9273003801642, 0.7701803567088483, 0.92194781914337, 0.7690628160942707, 0.7433110429384416, 0.5842195424875491, 0.953111794488, 0.6663405802785274, 0.9329793790565483]


Mean, STDev of RMSE: 3.215588811012966 1.9535149217076844
Mean, STDev of R^2: 0.8197869815299077 0.13103124329668592


In [58]:
########################### AdaBoostR2 Transfer Learning Housing #####################################################
from sklearn.ensemble import AdaBoostRegressor

print("Adaboost.R2 Transfer Learning")
print("-------------------------------------------")

r2scorelist_AdaTL_housing = []
rmselist_AdaTL_housing = []

kf = KFold(n_splits = 10)

for train_idx, test_idx in kf.split(housing_tgt_df):
        
    housing_train_df_X = housing_tgt_df.iloc[train_idx].loc[:, features_housing]
    housing_test_df_X = housing_tgt_df.iloc[test_idx][features_housing]
    housing_train_df_y = housing_tgt_df.iloc[train_idx].loc[:,target_housing]
    housing_test_df_y = housing_tgt_df.loc[test_idx][target_housing]
    
    housing_X_df = pd.concat([housing_source_df_X, housing_train_df_X], ignore_index = True)
    housing_y_df = pd.concat([housing_source_df_y, housing_train_df_y], ignore_index = True)

    housing_np_train_X = housing_X_df.to_numpy()
    housing_np_train_y = housing_y_df.to_numpy()

    housing_np_test_X = housing_test_df_X.to_numpy()
    housing_np_test_y = housing_test_df_y.to_numpy()

    housing_np_train_y_list = housing_np_train_y.ravel()
    housing_np_test_y_list = housing_np_test_y.ravel()


    model_AdaTL_housing = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 6), learning_rate = 0.1, n_estimators = 100) 
    model_AdaTL_housing.fit(housing_np_train_X, housing_np_train_y_list)

    y_pred_AdaTL_housing = model_AdaTL_housing.predict(housing_np_test_X)

    mse_AdaTL_housing = sqrt(mean_squared_error(housing_np_test_y, y_pred_AdaTL_housing))
    rmselist_AdaTL_housing.append(mse_AdaTL_housing)
        
    r2_score_AdaTL_housing = pearsonr(housing_np_test_y_list, y_pred_AdaTL_housing)
    r2_score_AdaTL_housing = (r2_score_AdaTL_housing[0])**2
    r2scorelist_AdaTL_housing.append(r2_score_AdaTL_housing)

print("RMSE List of Adaboost.R2:", rmselist_AdaTL_housing)
print("R^2 List of AdaboostR2:", r2scorelist_AdaTL_housing)

print("\n")

print("Mean, Stdev of RMSE:", statistics.mean(rmselist_AdaTL_housing), statistics.stdev(rmselist_AdaTL_housing))
print("Mean, Stdev of R^2:", statistics.mean(r2scorelist_AdaTL_housing), statistics.stdev(r2scorelist_AdaTL_housing))


print("-------------------------------------------")

Adaboost.R2 Transfer Learning
-------------------------------------------
RMSE List of Adaboost.R2: [3.08093208818374, 2.1387404887311665, 1.4568218936233506, 3.1149603921322124, 5.649134873022684, 2.019204145681526, 7.443647763089025, 2.773863640063722, 2.891185148078168, 1.9126907919331761]
R^2 List of AdaboostR2: [0.8876367480075871, 0.9105254644040658, 0.7649082994209835, 0.9192629947440659, 0.8606479246265066, 0.8178371107386858, 0.5822544847648489, 0.9477779258828463, 0.7337170468465634, 0.9510545290145113]


Mean, Stdev of RMSE: 3.248118122453877 1.8707814067465698
Mean, Stdev of R^2: 0.8375622528450665 0.11628537575902734
-------------------------------------------


In [59]:
################################## Gradient Boosting Regression Transfer Learning Housing #######################################
from sklearn.ensemble import GradientBoostingRegressor

print("Gradient Boosting Regression Transfer Learning")
print("-------------------------------------------")

r2scorelist_GBRTL_housing = []
rmselist_GBRTL_housing = []

kf = KFold(n_splits = 10)

for train_idx, test_idx in kf.split(housing_tgt_df):
        
    housing_train_df_X = housing_tgt_df.iloc[train_idx].loc[:, features_housing]
    housing_test_df_X = housing_tgt_df.iloc[test_idx][features_housing]
    housing_train_df_y = housing_tgt_df.iloc[train_idx].loc[:,target_housing]
    housing_test_df_y = housing_tgt_df.loc[test_idx][target_housing]
    
    housing_X_df = pd.concat([housing_source_df_X, housing_train_df_X], ignore_index = True)
    housing_y_df = pd.concat([housing_source_df_y, housing_train_df_y], ignore_index = True)

    housing_np_train_X = housing_X_df.to_numpy()
    housing_np_train_y = housing_y_df.to_numpy()

    housing_np_test_X = housing_test_df_X.to_numpy()
    housing_np_test_y = housing_test_df_y.to_numpy()

    housing_np_train_y_list = housing_np_train_y.ravel()
    housing_np_test_y_list = housing_np_test_y.ravel()


    model_GBRTL_housing = GradientBoostingRegressor(learning_rate = 0.1, max_depth = 6, n_estimators = 100, subsample=0.5)
    model_GBRTL_housing.fit(housing_np_train_X, housing_np_train_y_list)

    y_pred_GBRTL_housing = model_GBRTL_housing.predict(housing_test_df_X) ##Using dataframe instead of the numpy matrix

    mse_GBRTL_housing = sqrt(mean_squared_error(housing_np_test_y, y_pred_GBRTL_housing))
    rmselist_GBRTL_housing.append(mse_GBRTL_housing)
        
    r2_score_GBRTL_housing = pearsonr(housing_np_test_y_list, y_pred_GBRTL_housing)
    r2_score_GBRTL_housing = (r2_score_GBRTL_housing[0])**2
    r2scorelist_GBRTL_housing.append(r2_score_GBRTL_housing)

print("RMSE List of GBRTL:", rmselist_GBRTL_housing)
print("R^2 List of GBRTL:", r2scorelist_GBRTL_housing)

print("\n")


print("Mean, Stdev of RMSE:", statistics.mean(rmselist_GBRTL_housing), statistics.stdev(rmselist_GBRTL_housing))
print("Mean, Stdev of R^2:", statistics.mean(r2scorelist_GBRTL_housing), statistics.stdev(r2scorelist_GBRTL_housing))


print("-------------------------------------------")

Gradient Boosting Regression Transfer Learning
-------------------------------------------


/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(
/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(
/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(
/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(
/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(
/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/base.py:434

RMSE List of GBRTL: [3.3365898943988546, 1.9314218357565924, 1.4014541221659458, 2.663517238751539, 3.5138303728181417, 2.265241962785824, 5.527583102727759, 2.8479371972302503, 2.2317089245497987, 1.6815501564110624]
R^2 List of GBRTL: [0.8287449951602341, 0.8723717770521994, 0.6910027150241702, 0.9200721120477686, 0.9317614119565037, 0.7184981632632714, 0.8453889710001676, 0.956785475860066, 0.8346995545365778, 0.9000020175806475]


Mean, Stdev of RMSE: 2.740083480759577 1.1922167626207334
Mean, Stdev of R^2: 0.8499327193481606 0.08773623999474399
-------------------------------------------


/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(
/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(


In [71]:
from scipy import stats

stats_StAd_rmse, pval_StAd_rmse = stats.ttest_ind(rmselist_stradaboost_housing, rmselist_AdaTL_housing)
print(stats_StAd_rmse, pval_StAd_rmse)

stats_StGb_rmse, pval_StGb_rmse = stats.ttest_ind(rmselist_stradaboost_housing, rmselist_GBRTL_housing)
print(stats_StGb_rmse, pval_StGb_rmse)

stats_StTr_rmse, pval_StTr_rmse = stats.ttest_ind(rmselist_stradaboost_housing, rmselist_TwoTrAda_housing)
print(stats_StTr_rmse, pval_StTr_rmse)

print("\n")

stats_StAd_r2, pval_StAd_r2 = stats.ttest_ind(r2scorelist_stradaboost_housing, r2scorelist_AdaTL_housing)
print(stats_StAd_r2, pval_StAd_r2)

stats_StGb_r2, pval_StGb_r2 = stats.ttest_ind(r2scorelist_stradaboost_housing, r2scorelist_GBRTL_housing)
print(stats_StGb_r2, pval_StGb_r2)

stats_StTr_r2, pval_StTr_r2 = stats.ttest_ind(r2scorelist_stradaboost_housing, r2scorelist_TwoTrAda_housing)
print(stats_StTr_r2, pval_StTr_r2)

-0.11320807483759392 0.9111186613495608
0.6142376111006888 0.5467442730953394
-0.7877168016462025 0.4411126634078205


-0.3577646135896259 0.7246802877655424
-0.6235793100410496 0.5407261834520463
1.9792126669704677 0.06329652761434805


In [74]:
#### Dependent(paired) t-test
from scipy.stats import ttest_rel


stats_StAd_rmse, pval_StAd_rmse = stats.ttest_rel(rmselist_stradaboost_housing, rmselist_AdaTL_housing)
print(stats_StAd_rmse, pval_StAd_rmse)

stats_StGb_rmse, pval_StGb_rmse = stats.ttest_rel(rmselist_stradaboost_housing, rmselist_GBRTL_housing)
print(stats_StGb_rmse, pval_StGb_rmse)

stats_StTr_rmse, pval_StTr_rmse = stats.ttest_rel(rmselist_stradaboost_housing, rmselist_TwoTrAda_housing)
print(stats_StTr_rmse, pval_StTr_rmse)

print("\n")

stats_StAd_r2, pval_StAd_r2 = stats.ttest_rel(r2scorelist_stradaboost_housing, r2scorelist_AdaTL_housing)
print(stats_StAd_r2, pval_StAd_r2)

stats_StGb_r2, pval_StGb_r2 = stats.ttest_rel(r2scorelist_stradaboost_housing, r2scorelist_GBRTL_housing)
print(stats_StGb_r2, pval_StGb_r2)

stats_StTr_r2, pval_StTr_r2 = stats.ttest_rel(r2scorelist_stradaboost_housing, r2scorelist_TwoTrAda_housing)
print(stats_StTr_r2, pval_StTr_r2)

-0.7996854083202125 0.44448619214735663
1.7669802852038805 0.11103400281068965
-1.7148762764672583 0.12050921388855486


-1.1582914763418435 0.2765565179145322
-0.9686871369054251 0.35801096964238777
2.7169621385318274 0.023725511885053154


In [6]:
######################################################## Automobile ################################################################
## horsepower column has correlation 0.4 :: [46 - 230] :: 30
#################################################################################################################################
dropcol_initial_auto = ['name']
AutoData_df = pd.read_csv('UCI_regression/MPG/Auto.csv') ## horsepower column has correlation 0.4 :: [46 - 230] :: 30
AutoData_df = AutoData_df.drop(dropcol_initial_auto, axis = 1)
print("The shape of the Input data is: ", AutoData_df.shape)

drop_col_auto = ['horsepower']

auto_tgt_df = AutoData_df.loc[(AutoData_df['horsepower'] <= 80)]
auto_tgt_df = auto_tgt_df.drop(drop_col_auto, axis = 1)
auto_tgt_df = auto_tgt_df.reset_index(drop=True)
print("Target Set: ",auto_tgt_df.shape)

auto_source1_df = AutoData_df.loc[(AutoData_df['horsepower'] > 80) & (AutoData_df['horsepower'] <= 110)]
auto_source1_df = auto_source1_df.drop(drop_col_auto, axis = 1)
auto_source1_df = auto_source1_df.reset_index(drop=True)
print("Source Set 1: ",auto_source1_df.shape)

auto_source2_df = AutoData_df.loc[(AutoData_df['horsepower'] > 110)]
auto_source2_df = auto_source2_df.drop(drop_col_auto, axis = 1)
auto_source2_df = auto_source2_df.reset_index(drop=True)
print("Source Set 2: ",auto_source2_df.shape)

################################# Standardization ############################
from sklearn.preprocessing import StandardScaler
auto_cols = auto_tgt_df.columns.difference(['medv'])

ss = StandardScaler()
auto_tgt_df[auto_cols] = ss.fit_transform(auto_tgt_df[auto_cols])
auto_source1_df[auto_cols] = ss.fit_transform(auto_source1_df[auto_cols])
auto_source2_df[auto_cols] = ss.fit_transform(auto_source2_df[auto_cols])

## Concatenating the source datasets
auto_source_df = pd.concat([auto_source1_df, auto_source2_df], ignore_index = True)
auto_source_df = auto_source_df.reset_index(drop = True)

#################### Splitting into features and target ####################
target_auto = ['mpg']

auto_source_df_y = auto_source_df[target_auto]
auto_source_df_X = auto_source_df.drop(target_auto, axis = 1)

features_auto = auto_source_df_X.columns

The shape of the Input data is:  (392, 8)
Target Set:  (119, 7)
Source Set 1:  (157, 7)
Source Set 2:  (116, 7)


In [63]:
########################### TwoStage-TrAdaBoostR2 Auto #######################################

from adapt.instance_based import TrAdaBoost, TrAdaBoostR2, TwoStageTrAdaBoostR2


def get_estimator(**kwargs):
    return DecisionTreeRegressor(max_depth = 6)

kwargs_TwoTrAda = {'steps': 30,
                    'fold': 10,
                  'learning_rate': 0.1}

###########################################################################################################################

print("Two-Stage TrAdaboost.R2")
print("-------------------------------------------")

r2scorelist_TwoTrAda_auto = []
rmselist_TwoTrAda_auto = []

kf = KFold(n_splits = 10)


for train_idx, test_idx in kf.split(auto_tgt_df):
        
    auto_train_df_X = auto_tgt_df.iloc[train_idx].loc[:, features_auto]
    auto_test_df_X = auto_tgt_df.iloc[test_idx][features_auto]
    auto_train_df_y = auto_tgt_df.iloc[train_idx].loc[:,target_auto]
    auto_test_df_y = auto_tgt_df.loc[test_idx][target_auto]
    
    auto_X_df = pd.concat([auto_source_df_X, auto_train_df_X], ignore_index=True)
    auto_y_df = pd.concat([auto_source_df_y, auto_train_df_y], ignore_index=True)

    auto_np_train_X = auto_X_df.to_numpy()
    auto_np_train_y = auto_y_df.to_numpy()

    auto_np_test_X = auto_test_df_X.to_numpy()
    auto_np_test_y = auto_test_df_y.to_numpy()

    auto_np_train_y_list = auto_np_train_y.ravel()
    auto_np_test_y_list = auto_np_test_y.ravel()

    src_size_auto = len(auto_source_df_y)
    tgt_size_auto = len(auto_train_df_y)
    
    src_idx_auto = np.arange(start=0, stop=(src_size_auto - 1), step=1)
    tgt_idx_auto = np.arange(start=src_size_auto, stop=((src_size_auto + tgt_size_auto)-1), step=1)


    model_TwoTrAda_auto = TwoStageTrAdaBoostR2(get_estimator = get_estimator, n_estimators = 100, cv = 10) 
    model_TwoTrAda_auto.fit(auto_np_train_X, auto_np_train_y_list, src_idx_auto, tgt_idx_auto)

    y_pred_TwoTrAda_auto = model_TwoTrAda_auto.predict(auto_np_test_X)

    mse_TwoTrAda_auto = sqrt(mean_squared_error(auto_np_test_y, y_pred_TwoTrAda_auto))
    rmselist_TwoTrAda_auto.append(mse_TwoTrAda_auto)
        
    r2_score_TwoTrAda_auto = pearsonr(auto_np_test_y_list, y_pred_TwoTrAda_auto)
    r2_score_TwoTrAda_auto = (r2_score_TwoTrAda_auto[0])**2
    r2scorelist_TwoTrAda_auto.append(r2_score_TwoTrAda_auto)


print("RMSE List of TrAdaboost.R2:", rmselist_TwoTrAda_auto)
print("R^2 List of TrAdaboost.R2:", r2scorelist_TwoTrAda_auto)

print("\n")

print("Mean, Stdev of RMSE:", statistics.mean(rmselist_TwoTrAda_auto), statistics.stdev(rmselist_TwoTrAda_auto))
print("Mean, Stdev of R^2:", statistics.mean(r2scorelist_TwoTrAda_auto), statistics.stdev(r2scorelist_TwoTrAda_auto))


print("-------------------------------------------")

RMSE List of TrAdaboost.R2: [0.4243043188387373, 0.517195423770981, 0.6519664186500195, 0.31278770618508533, 0.9106076952664754, 0.7497905408530762, 0.7627149189699569, 1.121907562688717, 0.5554295196299659, 0.7564328904509023]
R^2 List of TrAdaboost.R2: [0.6554314111862946, 0.692713134420256, 0.6919383600901492, 0.8320631128745823, 0.12787056044070017, 0.6518615219543017, 0.3360064656178717, 0.0010568568496436836, 0.015296602690126363, 0.08681036876262815]


Mean, Stdev of RMSE: 0.6763136995303917 0.23779424942091276
Mean, Stdev of R^2: 0.4091048394886554 0.32809681066253876
-------------------------------------------


In [7]:
#################################### STrAdaBoost.R2 Active Sampling Auto ################################################################
from two_TrAdaBoostR2 import TwoStageTrAdaBoostR2

n_estimators = 100
steps = 30
fold = 10
random_state = np.random.RandomState(1)

r2scorelist_stradaboost_auto = []
rmselist_stradaboost_auto = []

print("Two-Stage STrAdaboost.R2")
print("-------------------------------------------")

kf = KFold(n_splits = 10)


for train_idx, test_idx in kf.split(auto_tgt_df):

    auto_train_df_X = auto_tgt_df.iloc[train_idx].loc[:, features_auto]
    auto_test_df_X = auto_tgt_df.iloc[test_idx][features_auto]
    auto_train_df_y = auto_tgt_df.iloc[train_idx].loc[:,target_auto]
    auto_test_df_y = auto_tgt_df.loc[test_idx][target_auto]

    auto_X_df = pd.concat([auto_source_df_X, auto_train_df_X], ignore_index=True)
    auto_y_df = pd.concat([auto_source_df_y, auto_train_df_y], ignore_index=True)

    auto_np_train_X = auto_X_df.to_numpy()
    auto_np_train_y = auto_y_df.to_numpy()

    auto_np_test_X = auto_test_df_X.to_numpy()
    auto_np_test_y = auto_test_df_y.to_numpy()

    auto_np_train_y_list = auto_np_train_y.ravel()
    auto_np_test_y_list = auto_np_test_y.ravel()

    sample_size = [len(auto_source_df_X), len(auto_train_df_X)]


    model_stradaboost_auto = TwoStageTrAdaBoostR2(DecisionTreeRegressor(max_depth = 6),
                        n_estimators = n_estimators, sample_size = sample_size,
                        steps = steps, fold = fold, random_state = random_state)


    model_stradaboost_auto.fit(auto_np_train_X, auto_np_train_y_list)
    y_pred_stradaboost_auto = model_stradaboost_auto.predict(auto_np_test_X)


    mse_stradaboost_auto = sqrt(mean_squared_error(auto_np_test_y, y_pred_stradaboost_auto))
    rmselist_stradaboost_auto.append(mse_stradaboost_auto)

    r2_score_stradaboost_auto = pearsonr(auto_np_test_y_list, y_pred_stradaboost_auto)
    r2_score_stradaboost_auto = (r2_score_stradaboost_auto[0])**2
    r2scorelist_stradaboost_auto.append(r2_score_stradaboost_auto)


print("RMSE List of STrAdaboostR2:", rmselist_stradaboost_auto)
print("R^2 List of STrAdaboostR2:", r2scorelist_stradaboost_auto)

print("\n")

print("Mean, Stdev of RMSE:", statistics.mean(rmselist_stradaboost_auto), statistics.stdev(rmselist_stradaboost_auto))
print("Mean, Stdev of R^2:", statistics.mean(r2scorelist_stradaboost_auto), statistics.stdev(r2scorelist_stradaboost_auto))


print("-------------------------------------------")

Two-Stage STrAdaboost.R2
-------------------------------------------
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
Inside Updated Sampling TrAdaBoost.R2
RMSE List of STrAdaboostR2: [0.5582854151910699, 0.4436385935251228, 0.6053261547923454, 0.45073472858445096, 0.9299787194313835, 0.4053955864341286, 0.9734016786434954, 1.0113509646269083, 0.30055525875003813, 0.506236630402056]
R^2 List of STrAdaboostR2: [0.8264871364575337, 0.5535348790808966, 0.8478100005728715, 0.8284247951938495, 0.04558676968026823, 0.5657909125356079, 0.3070933003253697, 0.05651190865072696, 0.5732291139313791, 0.5207809014653438]


Mean, Stdev of RMSE: 0.6184903730381 0.2579691048590702
Mean, Stdev of R^2: 0.51252497178

In [65]:
########################### AdaBoostR2 Transfer Learning Auto #####################################################

from sklearn.ensemble import AdaBoostRegressor


print("Adaboost.R2 Transfer Learning")
print("-------------------------------------------")

r2scorelist_AdaTL_auto = []
rmselist_AdaTL_auto = []

kf = KFold(n_splits = 10)

for train_idx, test_idx in kf.split(auto_tgt_df):
        
    auto_train_df_X = auto_tgt_df.iloc[train_idx].loc[:, features_auto]
    auto_test_df_X = auto_tgt_df.iloc[test_idx][features_auto]
    auto_train_df_y = auto_tgt_df.iloc[train_idx].loc[:,target_auto]
    auto_test_df_y = auto_tgt_df.loc[test_idx][target_auto]
    
    auto_X_df = pd.concat([auto_source_df_X, auto_train_df_X], ignore_index = True)
    auto_y_df = pd.concat([auto_source_df_y, auto_train_df_y], ignore_index = True)

    auto_np_train_X = auto_X_df.to_numpy()
    auto_np_train_y = auto_y_df.to_numpy()

    auto_np_test_X = auto_test_df_X.to_numpy()
    auto_np_test_y = auto_test_df_y.to_numpy()

    auto_np_train_y_list = auto_np_train_y.ravel()
    auto_np_test_y_list = auto_np_test_y.ravel()


    model_AdaTL_auto = AdaBoostRegressor(DecisionTreeRegressor(max_depth = 6), learning_rate = 0.1, n_estimators = 100) 
    model_AdaTL_auto.fit(auto_np_train_X, auto_np_train_y_list)

    y_pred_AdaTL_auto = model_AdaTL_auto.predict(auto_np_test_X)

    mse_AdaTL_auto = sqrt(mean_squared_error(auto_np_test_y, y_pred_AdaTL_auto))
    rmselist_AdaTL_auto.append(mse_AdaTL_auto)
        
    r2_score_AdaTL_auto = pearsonr(auto_np_test_y_list, y_pred_AdaTL_auto)
    r2_score_AdaTL_auto = (r2_score_AdaTL_auto[0])**2
    r2scorelist_AdaTL_auto.append(r2_score_AdaTL_auto)

print("RMSE List of TL Adaboost.R2:", rmselist_AdaTL_auto)
print("R^2 List of TL Adaboost.R2:", r2scorelist_AdaTL_auto)

print("\n")


print("Mean, Stdev of RMSE:", statistics.mean(rmselist_AdaTL_auto), statistics.stdev(rmselist_AdaTL_auto))
print("Mean, Stdev of R^2:", statistics.mean(r2scorelist_AdaTL_auto), statistics.stdev(r2scorelist_AdaTL_auto))


print("-------------------------------------------")

Adaboost.R2 Transfer Learning
-------------------------------------------
RMSE List of TL Adaboost.R2: [0.5135340662862089, 0.45359876137148253, 0.5592345186096519, 0.3134179943083618, 0.8944293662623398, 0.397764732639691, 0.9622384274167044, 0.9788976778450925, 0.30320142517921744, 0.48423314234349674]
R^2 List of TL Adaboost.R2: [0.8274059627537894, 0.6488743694737944, 0.8217234574146408, 0.8640250561996212, 0.10495483046155453, 0.5214274608220493, 0.4256364319788598, 0.0587711740828452, 0.5886460061348928, 0.5412400077405197]


Mean, Stdev of RMSE: 0.5860550112262247 0.2613246951608417
Mean, Stdev of R^2: 0.5402704757062567 0.2819481525530993
-------------------------------------------


In [66]:
######################### Gradient Boosting Regression Transfer Learning Auto #######################################
from sklearn.ensemble import GradientBoostingRegressor

print("Gradient Boosting Regression Transfer Learning")
print("-------------------------------------------")

r2scorelist_GBRTL_auto = []
rmselist_GBRTL_auto = []

kf = KFold(n_splits = 10)


for train_idx, test_idx in kf.split(auto_tgt_df):
        
    auto_train_df_X = auto_tgt_df.iloc[train_idx].loc[:, features_auto]
    auto_test_df_X = auto_tgt_df.iloc[test_idx][features_auto]
    auto_train_df_y = auto_tgt_df.iloc[train_idx].loc[:,target_auto]
    auto_test_df_y = auto_tgt_df.loc[test_idx][target_auto]
    
    auto_X_df = pd.concat([auto_source_df_X, auto_train_df_X], ignore_index = True)
    auto_y_df = pd.concat([auto_source_df_y, auto_train_df_y], ignore_index = True)

    auto_np_train_X = auto_X_df.to_numpy()
    auto_np_train_y = auto_y_df.to_numpy()

    auto_np_test_X = auto_test_df_X.to_numpy()
    auto_np_test_y = auto_test_df_y.to_numpy()

    auto_np_train_y_list = auto_np_train_y.ravel()
    auto_np_test_y_list = auto_np_test_y.ravel()


    model_GBRTL_auto = GradientBoostingRegressor(learning_rate = 0.1, max_depth = 6, n_estimators = 100, subsample=0.5)
    model_GBRTL_auto.fit(auto_np_train_X, auto_np_train_y_list)

    y_pred_GBRTL_auto = model_GBRTL_auto.predict(auto_test_df_X) ##Using dataframe instead of the numpy matrix

    mse_GBRTL_auto = sqrt(mean_squared_error(auto_np_test_y, y_pred_GBRTL_auto))
    rmselist_GBRTL_auto.append(mse_GBRTL_auto)
        
    r2_score_GBRTL_auto = pearsonr(auto_np_test_y_list, y_pred_GBRTL_auto)
    r2_score_GBRTL_auto = (r2_score_GBRTL_auto[0])**2
    r2scorelist_GBRTL_auto.append(r2_score_GBRTL_auto)

print("RMSE List of GBRTL:", rmselist_GBRTL_auto)
print("R^2 List of GBRTL:", r2scorelist_GBRTL_auto)

print("\n")

    
print("Mean, RMSE of GBRTL:", statistics.mean(rmselist_GBRTL_auto), statistics.stdev(rmselist_GBRTL_auto))
print("R^2 of GBRTL:", statistics.mean(r2scorelist_GBRTL_auto), statistics.stdev(r2scorelist_GBRTL_auto))


print("-------------------------------------------")

Gradient Boosting Regression Transfer Learning
-------------------------------------------


/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(
/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(
/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(
/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(
/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(
/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/base.py:434

RMSE List of GBRTL: [0.4799244385805726, 0.3587433734146151, 0.5365625957008113, 0.33479450177460846, 0.7964150001985001, 0.33743326679686797, 0.7912499860574775, 0.8972658678878713, 0.36048449001455546, 0.5728416221664219]
R^2 List of GBRTL: [0.8541224062235835, 0.6386941888664273, 0.7540019820483619, 0.7868740060520802, 0.14682011249376833, 0.6124702169484553, 0.45062813434510973, 0.08525885072998268, 0.5428910694971595, 0.36585768008897507]


Mean, RMSE of GBRTL: 0.5465715142592302 0.21317808988559206
R^2 of GBRTL: 0.5237618647293903 0.2619315829208315
-------------------------------------------


/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(
/Users/shrey/anaconda3/lib/python3.8/site-packages/sklearn/base.py:434: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(


In [72]:
#### Independent t-test
from scipy import stats

stats_StAd_rmse, pval_StAd_rmse = stats.ttest_ind(rmselist_stradaboost_auto, rmselist_AdaTL_auto)
print(stats_StAd_rmse, pval_StAd_rmse)

stats_StGb_rmse, pval_StGb_rmse = stats.ttest_ind(rmselist_stradaboost_auto, rmselist_GBRTL_auto)
print(stats_StGb_rmse, pval_StGb_rmse)

stats_StTr_rmse, pval_StTr_rmse = stats.ttest_ind(rmselist_stradaboost_auto, rmselist_TwoTrAda_auto)
print(stats_StTr_rmse, pval_StTr_rmse)

print("\n")

stats_StAd_r2, pval_StAd_r2 = stats.ttest_ind(r2scorelist_stradaboost_auto, r2scorelist_AdaTL_auto)
print(stats_StAd_r2, pval_StAd_r2)

stats_StGb_r2, pval_StGb_r2 = stats.ttest_ind(r2scorelist_stradaboost_auto, r2scorelist_GBRTL_auto)
print(stats_StGb_r2, pval_StGb_r2)

stats_StTr_r2, pval_StTr_r2 = stats.ttest_ind(r2scorelist_stradaboost_auto, r2scorelist_TwoTrAda_auto)
print(stats_StTr_r2, pval_StTr_r2)

0.09387925740000946 0.9262421791063029
0.468273988433133 0.6452092125474611
-0.7005342904293791 0.4925503891649965


-0.21561187287426226 0.8317146184323403
-0.08862966216384667 0.9303548911060949
0.7542751148496798 0.46043871525449653


In [73]:
#### Dependent(paired) t-test
from scipy.stats import ttest_rel

stats_StAd_rmse, pval_StAd_rmse = stats.ttest_rel(rmselist_stradaboost_auto, rmselist_AdaTL_auto)
print(stats_StAd_rmse, pval_StAd_rmse)

stats_StGb_rmse, pval_StGb_rmse = stats.ttest_rel(rmselist_stradaboost_auto, rmselist_GBRTL_auto)
print(stats_StGb_rmse, pval_StGb_rmse)

stats_StTr_rmse, pval_StTr_rmse = stats.ttest_rel(rmselist_stradaboost_auto, rmselist_TwoTrAda_auto)
print(stats_StTr_rmse, pval_StTr_rmse)

print("\n")

stats_StAd_r2, pval_StAd_r2 = stats.ttest_rel(r2scorelist_stradaboost_auto, r2scorelist_AdaTL_auto)
print(stats_StAd_r2, pval_StAd_r2)

stats_StGb_r2, pval_StGb_r2 = stats.ttest_rel(r2scorelist_stradaboost_auto, r2scorelist_GBRTL_auto)
print(stats_StGb_r2, pval_StGb_r2)

stats_StTr_r2, pval_StTr_r2 = stats.ttest_rel(r2scorelist_stradaboost_auto, r2scorelist_TwoTrAda_auto)
print(stats_StTr_r2, pval_StTr_r2)

1.027595320206241 0.33096466564742744
2.3507126938039717 0.04325455224503656
-1.4188348219534268 0.1896360652668623


-1.5799623613158462 0.14857296728108557
-0.55014538292201 0.5956072149748616
1.6768498418527644 0.12788653793474053


In [ ]:
import matplotlib.pyplot as plt


labels = ['Concrete', 'Housing', 'Auto']

TR_means = [20, 35, 30]
STR_means = [25, 32, 34]
ADA_means = [25, 32, 34]
GBR_means = [25, 32, 34]

TR_std = [20, 35, 30]
STR_std = [25, 32, 34]
ADA_std = [25, 32, 34]
GBR_std = [25, 32, 34]

width = 0.35       

fig, ax = plt.subplots()

ax.bar(labels, men_means, width, yerr=men_std, label='Men')
ax.bar(labels, women_means, width, yerr=women_std, bottom=men_means,
       label='Women')

ax.set_ylabel('Scores')
ax.set_title('Scores by group and gender')
ax.legend()

plt.show()